In [1]:
##
# Una clase que puede ser utilizada para leer los datos del PDF de "Specialised Testing Laboratory (Pty) ltd"
##

# Se instala "tabula-py" para leer tablas de PDF a dataframes de pandas
%pip install tabula-py
import tabula
import pandas as pd
from pathlib import Path


class PDFReader:
    """Lee los datos de las tablas en el PDF dado"""

    # Cada página de este PDF tiene multiples tablas, estas son:
    #
    # 0 -> Cabecera de la página
    # 1 -> PSD con cabecera de datos
    # 2 -> Atterberg
    # 3 -> pH y conductividad
    # 4 -> MDD / OMC
    # 5 -> CBR
    # 6 -> UCS
    # 7 -> Clasificación COLTO
    # 8 -> Pie de la página
    PSD_INDEX = 1
    ATTERBERG_INDEX = 2
    DATA_HEADER_INDEX = 1

    def __init__(self, filename):
        # Transponer las tablas las hace mas manejables, eliminar las fila vacias también
        self._filename = Path(filename).name
        self._dataframes = [
            df.T.dropna(axis=0, how="all")
            for df in tabula.read_pdf(filename, pandas_options={'header': None}, pages="all")
        ]
    
    def cant_paginas(self):
        """Calcula la cantidad de páginas para este tipo de PDFs"""
        return int(len(self._dataframes)/9)

    def _get_info_for_page(self, page_num):
        """Extrae los datos de los ensayos para la pagina dada. La primer página es la 0 (cero)"""
        # La tabla 1. no solo contiene los datos de PSD sino que también contiene la cabecera
        # común a todas las tablas de esa página.
        # Cada página posee diferente cabecera.
        return self._dataframes[self.DATA_HEADER_INDEX + (page_num * 9)].iloc[:, :3]

    def _concat_info(self, datos, info):
        """Concatena la info con los datos del ensayo y setea la primer fila como cabecera"""
        df = pd.concat([info, datos], axis=1)

        # Define la primer fila como cabecera, tambien elimina caracteres invalidos
        df.columns = [
            c.replace(" ", "_").replace("(", "").replace(")", "")
            for c in df.iloc[0, :]
        ]
        df = df.iloc[1:, :]

        # Agrega el nombre del archivo fuente de datos
        df.loc[:, "Data_filename"] = self._filename

        return df

    def get_ATTERBERG_data_by_page(self, page_num):
        """Devuelve la tabla con los datos del ensayo ATTERBERG para la página dada.
        La primer página es la 0 (cero)"""
        # Los datos de ATTERBERG
        d = self._dataframes[self.ATTERBERG_INDEX + (page_num * 9)]

        h = self._get_info_for_page(page_num)
        return self._concat_info(d, h)

    def get_all_ATTERBERG_data(self):
        """Devuelve todos los datos del ensayo ATTERBERG en el PDF"""
        all_psd = [
            self.get_ATTERBERG_data_by_page(i) for i in range(self.cant_paginas())
        ]

        return pd.concat(all_psd, axis=0, ignore_index=True)

    def get_PSD_data_by_page(self, page_num):
        """Devuelve la tabla con los datos del ensayo PSD para la página dada.
        La primer página es la 0 (cero)"""
        # Los datos de PSD
        d = self._dataframes[self.PSD_INDEX + (page_num * 9)].iloc[:, 3:]

        h = self._get_info_for_page(page_num)
        return self._concat_info(d, h)

    def get_all_PSD_data(self):
        """Devuelve todos los datos de los ensayos PSD en el PDF"""
        all_psd = [
            self.get_PSD_data_by_page(i) for i in range(self.cant_paginas())
        ]

        return pd.concat(all_psd, axis=0, ignore_index=True)


StatementMeta(, 19e3bbb3-27d1-4005-a733-747578278e70, 8, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 31.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [4]:
##
# Lee los datos de los ensayos del PDF a dataframes de pandas
##
PDF_FILENAME = "/lakehouse/default/Files/SRK-169-Summary Sheet.pdf"
reader = PDFReader(PDF_FILENAME)

psd = reader.get_all_PSD_data()
atter = reader.get_all_ATTERBERG_data()


StatementMeta(, 19e3bbb3-27d1-4005-a733-747578278e70, 12, Finished, Available, Finished)

FileNotFoundError: [Errno 2] No such file or directory: '/lakehouse/default/Files/SRK-169-Summary Sheet.pdf'

In [3]:
##
# Construye las tablas de datos: PSD y Atterberg
##

from pyspark.sql.types import StructType, StructField, StringType


# Indice de datos PSD
psd_schema = StructType([
    StructField("Sample", StringType(), nullable=False),
    StructField("Depth_m", StringType(), nullable=False),
    StructField("Lab_No", StringType(), nullable=False),
    StructField("53.0", StringType(), nullable=False),
    StructField("37.5", StringType(), nullable=False),
    StructField("26.5", StringType(), nullable=False),
    StructField("19.0", StringType(), nullable=False),
    StructField("13.2", StringType(), nullable=False),
    StructField("9.5", StringType(), nullable=False),
    StructField("6.7", StringType(), nullable=False),
    StructField("4.75", StringType(), nullable=False),
    StructField("2.0", StringType(), nullable=False),
    StructField("1.0", StringType(), nullable=False),
    StructField("0.425", StringType(), nullable=False),
    StructField("0.250", StringType(), nullable=False),
    StructField("0.150", StringType(), nullable=False),
    StructField("0.075", StringType(), nullable=False),
    StructField("0.060", StringType(), nullable=False),
    StructField("0.050", StringType(), nullable=False),
    StructField("0.035", StringType(), nullable=False),
    StructField("0.020", StringType(), nullable=False),
    StructField("0.006", StringType(), nullable=False),
    StructField("0.002", StringType(), nullable=False),
    StructField("GM", StringType(), nullable=False),
    StructField("Data_filename", StringType(), nullable=False)
])


# Indice de datos ATTERBERG
atterberg_schema = StructType([
    StructField("Sample", StringType(), nullable=False),
    StructField("Depth_m", StringType(), nullable=False),
    StructField("Lab_No", StringType(), nullable=False),
    StructField("LL_%", StringType(), nullable=False),
    StructField("PI_%", StringType(), nullable=False),
    StructField("LS_%", StringType(), nullable=False),
    StructField("Data_filename", StringType(), nullable=False)
])


# Crea las tablas si es que no existen ya
spark.createDataFrame([], psd_schema).write.format("delta").mode("ignore").saveAsTable("data_PSD")
spark.createDataFrame([], atterberg_schema).write.format("delta").mode("ignore").saveAsTable("data_atterberg")


StatementMeta(, 0b9c56c6-5559-42ff-acb4-f6f8d59bdede, 11, Finished, Available, Finished)

In [4]:
##
# Guarda los datos en las tablas
#
# Tiene controles para evitar guardar datos duplicados
##

from delta.tables import DeltaTable

def guardar_datos(datos, nombre_tabla, schema):
    """Guarga los datos en la tabla dada.
    Los datos duplicados no se guardan"""

    # Los datos son transformados a un Spark Dataframe
    nuevos_datos = spark.createDataFrame(datos, schema)

    # Cuando los campos NO coinciden se insertan los datos
    DeltaTable.forName(spark, nombre_tabla).alias("tabla").merge(
        nuevos_datos.alias("nuevo"),
        """
        tabla.Data_filename = nuevo.Data_filename
        AND
        tabla.Sample = nuevo.Sample
        AND
        tabla.Lab_No = nuevo.Lab_No
        """
    ).whenNotMatchedInsertAll().execute()


# Se guardan los datos
guardar_datos(psd, "data_PSD", psd_schema)
guardar_datos(atter, "data_atterberg", atterberg_schema)


StatementMeta(, 0b9c56c6-5559-42ff-acb4-f6f8d59bdede, 12, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Exception thrown when converting pandas.Series (object) with name 'LL_%' to Arrow Array (string).
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.


In [5]:
%%sql
select * from data_PSD;
select * from data_atterberg;


StatementMeta(, 0b9c56c6-5559-42ff-acb4-f6f8d59bdede, 14, Finished, Available, Finished)

<Spark SQL result set with 21 rows and 25 fields>

<Spark SQL result set with 21 rows and 7 fields>